# Run Sandwich Attacks on Swaps from the Public Mempool

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

current_path = sys.path[0]
sys.path.append(
    current_path[: current_path.find("defi-measurement")]
    + "liquidity-distribution-history"
)

sys.path.append("..")


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy
from dotenv import load_dotenv
from matplotlib.ticker import MaxNLocator
from pool_state import v3Pool
from sqlalchemy import create_engine
from tqdm import tqdm
from collections import deque

load_dotenv(override=True)
from experiments.random_permutations import load_pool

from decimal import getcontext

getcontext().prec = 100  # Set the precision high enough for our purposes


# Read in the environment variables
postgres_uri_mp = os.environ["POSTGRESQL_URI_MP"]
postgres_uri_us = os.environ["POSTGRESQL_URI_US"]

engine = create_engine(postgres_uri_mp)

## Get the Data

In [3]:
query = """
SELECT *
FROM SWAP_LIMIT_PRICE AS LIM
INNER JOIN MEMPOOL_TRANSACTIONS AS MEM ON LIM.CALL_TX_HASH = MEM.HASH
"""

col_rename = dict(
    call_block_number='block_number',
    contract_address='pool',
)

df = pd.read_sql_query(query, engine).rename(columns=col_rename).sort_values(by=['pool', 'block_number'])

df.shape

(828111, 14)

In [4]:
swap_counts = df.groupby(by=['pool', 'block_number'])[['call_success']].count().sort_values('call_success', ascending=False)

swap_counts[swap_counts == 1].call_success.sum(), swap_counts[swap_counts > 1].call_success.sum(), swap_counts.call_success.sum()

(692368.0, 135743.0, 828111)

## Create Sandwich Attacks on Single Swaps

Start with this to validate the approach.

In [5]:
single_swap_blocks = swap_counts[swap_counts == 1].sort_index()

# Make into a dictionary of the form {block: [block_number]}
single_swap_blocks = single_swap_blocks.reset_index()
single_swap_blocks = single_swap_blocks.groupby('pool')['block_number'].apply(list).to_dict()

single_swap_blocks = dict(sorted(single_swap_blocks.items(), key=lambda item: len(item[1]), reverse=True))

In [6]:
def get_mev_single():
    pass

In [16]:
# Just starting to load some pools into the cache to speed up the process down the line
for pool_addr, blocks in tqdm(single_swap_blocks.items()):
    try:
        pool = load_pool(pool_addr, postgres_uri_us, verbose=False)
    except AssertionError:
        print(f"Pool {pool_addr} not found in the database")
        continue

  0%|          | 0/3067 [00:00<?, ?it/s]

In [15]:
for pool_addr, blocks in single_swap_blocks.items():
    print(f"Pool: {pool_addr}")
    # pool = load_pool(pool_addr, postgres_uri_us)

    for block in blocks:

        swap = df[(df.pool == pool_addr) & (df.block_number == block)]
        assert len(swap) == 1, f"Expected 1 swap, got {len(swap)}"

        swap = swap.iloc[0]

        print(swap.dtypes)

        swap_params =         {
            "input": int(swap.amountSpecified),
            "tokenIn": pool.token0 if swap.zeroForOne else pool.token1,
            "as_of": swap.block_number,
            "gasFee": True,
        }

        output, heur = pool.swap_in(swap_params, )


        print(f"Swap ({swap.amountSpecified} for {output})")
        print(f"Next sqrt price: {heur.sqrtP_next} vs. sqrtPriceLimit: {swap.sqrtPriceLimitX96}")


Pool: 0x11b815efb8f581194ae79006d24e0d814b7697f6
object
Swap (1961174914507662484 for 3675671813)
Next sqrt price: 4.330289341883109e-05 vs. sqrtPriceLimit: 4295128740
object
Swap (147050485429 for 61294088165826316401298985296330752)
Next sqrt price: -2679.3742477103174 vs. sqrtPriceLimit: 3431989699364555050713088
object
Swap (1060000000 for 564618674187906368)
Next sqrt price: 4.3317883458019664e-05 vs. sqrtPriceLimit: 1461446703485210103287273052203988822378723970341
object
Swap (-2000000000 for 3)
Next sqrt price: 4.331788345802003e-05 vs. sqrtPriceLimit: 4295128740
object
Swap (580144155260846821 for 1088023731)
Next sqrt price: 4.331709211496837e-05 vs. sqrtPriceLimit: 4295128740
object
Swap (145000000000000000 for 271937515)
Next sqrt price: 4.3317065713542496e-05 vs. sqrtPriceLimit: 4295128740
object
Swap (5010000000 for 2668677726411982336)
Next sqrt price: 4.3317624651417274e-05 vs. sqrtPriceLimit: 1461446703485210103287273052203988822378723970341
object
Swap (985000000 for 

AssertionError: Not enough liquidity in pool